# Santander customer satisfaction: model selection

Using hyperopt to define search spaces and optimize hyperparameters. 

## Imports and code

Importing the base code that contains many functions, wrappers for algorithms, cross validation and model selection frameworks.

In [1]:
# starting up a console attached to this kernel
%matplotlib inline
%qtconsole
import os

# importing base code
os.chdir('/home/guilherme/Documents/Kaggle/santander-satisfaction/code')
from base import *

# changing to competition dir
os.chdir('/home/guilherme/Documents/Kaggle/santander-satisfaction')

# Model selection frameworks and search spaces

Fitting algorithms on raw data. Hyperparameter optimization is done using hyperopt (Tree of Parzen Estimators). 

## XGBoost

* **1st-round:** 
    * less expensive 10-fold CV
    * choosing best data 
    * tweaking class weights
    * univariate feature selection
    * no feature expansion

* **2nd-round:** 
    * more expensive 7-fold with 5 repetitions CV
    * improve on first round

* **3rd-round:**
    * over and undersampling -> worse! stopped test
    * stability selection

* **4th-round:**
    * focus on feature engineering

In [4]:
# search space for hyperparameter optimization
xgb_space = {'model': xgb.XGBClassifier,
             'params': {'n_estimators' : hp.normal('xgb_n', 500, 100),
                        'learning_rate' : hp.uniform('xgb_eta', 0.01, 0.03),
                        'max_depth' : hp.quniform('xgb_max_depth', 2, 8, 1),
                        'min_child_weight' : hp.quniform('xgb_min_child_weight', 1, 6, 1),
                        'subsample' : hp.uniform('xgb_subsample', 0.8, 1),
                        'gamma' : hp.uniform('xgb_gamma', 0.0, 0.4),
                        'colsample_bytree' : hp.uniform('xgb_colsample_bytree', 0.2, 0.8),
                        'objective': hp.choice('xgb_obj', ['binary:logistic']),
                        'scale_pos_weight': hp.uniform('xgb_w', 1.0, 4.0)
                        },
             'preproc': {'na_input': {'strategy': 'mean'},
                         'var_thres': {'threshold': 0.0},
                         'sel_perc': {'score_func': False,
                                      'percentile': False}
                        },
             'resmpl': hp.choice('resmpl', [{'method': False, 'params': False}]),
             'data': hp.choice('dc',[{'real': 'data/selected/st-train.csv',
                                      'cat': None, 
                                      'ground-truth': 'data/target.csv'}]),                
             'feat_exp': {'n': 0}, #hp.quniform('exp_n', 0, 100, 20)
             'fit_params': {'eval_metric': 'auc'},
             'y_transf': hp.choice('trf', [None]),
            }

# model selection
eval_number = 0
trials = Trials()      
best_xgb = optimize(framework, xgb_space, 20, trials)

# saving trials
save_obj(trials, 'trials/4th-round')

eval_number: 1 {'feat_exp': {'n': 0}, 'fit_params': {'eval_metric': 'auc'}, 'preproc': {'var_thres': {'threshold': 0.0}, 'na_input': {'strategy': 'mean'}, 'sel_perc': {'percentile': False, 'score_func': False}}, 'y_transf': None, 'resmpl': {'params': False, 'method': False}, 'params': {'colsample_bytree': 0.48855914089061664, 'scale_pos_weight': 3.089407556793585, 'learning_rate': 0.023696594771697263, 'min_child_weight': 5.0, 'n_estimators': 528.2978498051992, 'subsample': 0.8572278669900759, 'objective': 'binary:logistic', 'max_depth': 5.0, 'gamma': 0.3923056793538462}, 'model': <class 'xgboost.sklearn.XGBClassifier'>, 'data': {'real': 'data/selected/st-train.csv', 'ground-truth': 'data/target.csv', 'cat': None}}
cv underway.......round 0 AUC: 0.8373 | average so far: 0.8373 +- 0.0041
cv underway

KeyboardInterrupt: 

# Fit Analysis

Checking evolution of trials, residuals and overall fit.

## Visualizing trials

In [ ]:
# loading trials
trials = load_obj('trials/3rd-round')

# reading R-squared
r2 = [trials.trials[i]['result']['auc_avg'] for i in range(len(trials.trials))]
r2_std = [trials.trials[i]['result']['auc_std'] for i in range(len(trials.trials))]

# plotting trials results
plt.figure(figsize=[15,10])
plt.plot(range(len(r2)), r2, 'ko')
plt.xlabel('Trial number')
plt.ylabel('AUC')
plt.errorbar(range(len(r2)), r2, yerr=r2_std, fmt='ko')
plt.show()

## Sorted trials

In [ ]:
# loading trials
trials = load_obj('trials/3rd-round')

# getting top 10 trials
top = [get_best(trials, i) for i in range(len(trials.trials))]

r2 = [top[i]['result']['auc_avg'] for i in range(len(top))]
r2_std = [top[i]['result']['auc_std'] for i in range(len(top))]

# plotting trials results
plt.figure(figsize=[15,10])
plt.plot(range(len(r2)), r2, 'ko')
plt.axis([-1, len(trials.trials), 0.76, 0.86])
plt.errorbar(range(len(r2)), r2, yerr=r2_std, fmt='ko')
plt.show()

## Hyperparameter importance

Let us plot the influence of each hyperparameter.

In [ ]:
# loading trials
import seaborn as sns
trials = load_obj('trials/3rd-round')
top = [get_best(trials, i) for i in range(len(trials.trials))]
result = [top[i]['result']['auc_avg'] for i in range(len(top))]

hyper_df = hypersummary(top, result, 'auc')

hyper_df

In [ ]:
g = sns.PairGrid(hyper_df, hue='auc')
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)
g.savefig('vis/hyperparameters/2nd-round.png')

## ROC Curves

Let us check the results in more detail.

In [ ]:
# loading trials
trials = load_obj('trials/3rd-round')
top = [get_best(trials, i) for i in range(len(trials.trials))]

# ranges
rngs = get_plot_ranges(1, 40)
    
# plot ROC for the best models
for i in range(len(rngs)):
    
    plt.figure(figsize=[18,10])
    count = 1
    for model in rngs[i]:
    
        # getting ith best model (0 is the best)
        space = get_best(trials, ind=model)
        plt.subplot(1, 1, count)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.title('Model {0} (mean AUC = {1:.3f}) '.format(model, space['result']['auc_avg']))
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])

        for fold in range(10):

            # computing roc
            probs = space['result']['results']['folds']['probs'][fold]
            gnd_truth = space['result']['results']['folds']['gnd_truth'][fold]
            fpr, tpr, thresholds = metrics.roc_curve(gnd_truth, probs)
            auc = metrics.roc_auc_score(gnd_truth, probs) 
                
            # plotting
            plt.plot(fpr, tpr, label='Fold {0} (area = {1:.3f})'.format(fold, auc))
            plt.legend(loc="lower right")
        
        count += 1
    
    # saving full plot
    directory = 'vis/roc_curves/2nd-round'
    if not os.path.exists(directory):
        os.makedirs(directory)
    print directory + '/roc-{}.png'.format(i)
    plt.savefig(directory + '/roc-{}.png'.format(i))
    plt.clf()

## Ensembling

Can we get better performance if we average model predictions?

Loading trials and model data:

In [ ]:
# loading trials
trials = load_obj('trials/3rd-round')
trials = [get_best(trials, i) for i in range(len(trials.trials))]

# extracting target
target_m = []
for r in range(5):
    fold_m = []
    for f in range(7):
        fold_m.append(np.array(trials[0]['result']['results'][r]['folds']['gnd_truth'][f]))
    target_m.append(fold_m)
# as numpy array
target_m = np.array(target_m)

# extracting model data
model_dict = {}
for m in range(40):
    model_dict[m] = {}
    probs_m = []
    auc = []
    for r in range(5):
        fold_m = []
        for f in range(7):
             fold_m.append(np.array(trials[m]['result']['results'][r]['folds']['probs'][f]))
        probs_m.append(fold_m)
        
    model_dict[m]['pm'] = probs_m
    model_dict[m]['auc'] = trials[m]['result']['auc_avg']

In [ ]:
def eval_7_5_fold(target_m, models, weights=None):
    probs = []
    for r in range(5):
        folds = []
        for f in range (7):
            folds.append([0]*len(target_m[r][f]))
        probs.append(folds)
    
    for i, model in enumerate(models):
        for r in range(5):
            for f in range (7):
                if weights == None:
                    probs[r][f] = probs[r][f] + model['pm'][r][f]*1/len(models)
                else:
                    probs[r][f] = probs[r][f] + model['pm'][r][f]*weights[i]/sum(weights)
    aucs = []
    for r in range(5):
        folds = []
        for f in range(7):
            folds.append(metrics.roc_auc_score(target_m[r][f], probs[r][f]))
        best = folds.index(max(folds))
        worst = folds.index(min(folds))
        folds = [v for i, v in enumerate(folds) if i not in [best, worst]]
        aucs.append(folds)
    return(np.mean(aucs))

## Simple average: greedy

Joining results of best models in a greedy manner.

In [ ]:
in_models = []
model_inds = []
max_score = 0
for m in range(40):
    
    score = eval_7_5_fold(target_m, in_models + [model_dict[m]])
    if score > max_score:
        max_score = score
        in_models.append(model_dict[m])
        model_inds.append(m)
        print 'AUC {0:.6f} for combination: {1}'.format(score, model_inds)
        

## Feature importances

Checking which ones are more important.

# Predictions

Predicting on test data.

In [ ]:
ens_preds = []
for i in [0, 1, 2, 4, 5, 10]:    
    
    # getting space
    trials = load_obj('trials/2nd-round')
    space = get_best(trials, i)['result']['parameters']

    # reading csv & casting into hashable type
    train_real = pd.read_csv(space['data']['real'])
    y_train = pd.read_csv(space['data']['ground-truth'])['TARGET']

    # same for test
    test_real = pd.read_csv(space['data']['real'].replace('train', 'test'))

    # feature expansion
    feat_exp = FeatureExpansion()
    train_real = feat_exp.transform(train_real, space['feat_exp']['op_log'])
    train_real = csr_matrix(train_real)
    test_real = feat_exp.transform(test_real, space['feat_exp']['op_log'])
    test_real = csr_matrix(test_real)
    
    # categorical data
    try:
        # train
        train_cat = load_obj(space['data']['cat'])
        train = hstack([train_real, train_cat]).tocsr()
        # test
        test_cat = load_obj(space['data']['cat'].replace('train', 'test'))
        test = hstack([test_real, test_cat]).tocsr()
    except:
        train = train_real.tocsr()
        test = test_real.tocsr()

    # casting values to int
    space['params']['n_estimators'] = int(space['params']['n_estimators'])
    space['preproc']['sel_perc']['percentile'] = int(space['preproc']['sel_perc']['percentile'])

    # model loaded in search space
    algo = space['model'](**space['params'])
    y_transform = TargetTransform(space['y_transf'])
    model = sklearn_wrapper(algo, space['preproc'], y_transform)

    # fitting and predicting
    model.fit(train, y_train, fit_params={'eval_metric': 'auc'})
    probs = model.predict_proba(test)
    ens_preds.append(probs[:,1])
    sys.stdout.write('{}, '.format(i))
    
# ids
ID_col = pd.read_csv('data/raw/test.csv')['ID']

# averaging predictions
ens_preds = np.mean(ens_preds, axis=0)
#weights = np.array([0.19339375, 0.92619747, 0.01661305,  0.39647386,  0.49869799,  0.31282152])
#new_preds = np.transpose(np.array(ens_preds)) * np.transpose(weights)
#new_preds = np.sum(new_preds, axis=1)/sum(weights)

# making a submission
sub = pd.DataFrame({'ID': np.array(ID_col).astype(int), 'TARGET': ens_preds})
sub.to_csv('submissions/sub11.csv', index=False)

## Submission log

* **sub7:** single xgb, 1st round best - CV: 0.842 | LB: 0.840010.
* **sub8:** ensemble of 20 best models of 1st round - CV: ? | LB: 0.841072.
* **sub9:** best model of 2nd round - CV: 0.842240 | LB: 0.840917
* **sub10:** greedy optimization ensemble of 2nd round models (output average) - CV: 0.842503 | LB: 0.841378
* **sub11:** optimization of weights of ensemble of 2nd round models (output weighted average) - CV: 0.842525 | LB:

## Rules of thumb

Some rules of thumb can be added in the submission, such as everyone under age 23 is happy. Let us see what happens.

In [20]:
preds = pd.read_csv('submissions/sub10.csv')['TARGET']
tc = pd.read_csv('data/no-duplicates/test.csv')

In [21]:
nv = tc['num_var33'] + tc['saldo_medio_var33_ult3'] + tc['saldo_medio_var44_hace2'] + tc['saldo_medio_var44_hace3'] + tc['saldo_medio_var33_ult1'] + tc['saldo_medio_var44_ult1']

preds[nv > 0][1] = 0
preds[tc['var15'] < 23] = 0
preds[tc['saldo_medio_var5_hace2'] > 160000] = 0
preds[tc['saldo_var33'] > 0] = 0
preds[tc['var38'] > 3988596] = 0
preds[tc['var21'] > 7500] = 0
preds[tc['num_var30'] > 9] = 0
preds[tc['num_var13_0'] > 6] = 0
preds[tc['num_var33_0'] > 0] = 0
preds[tc['imp_ent_var16_ult1'] > 51003] = 0
preds[tc['imp_op_var39_comer_ult3'] > 13184] = 0
preds[tc['saldo_medio_var5_ult3'] > 108251] = 0

ID_col = tc['ID']

# making a submission
sub = pd.DataFrame({'ID': np.array(ID_col).astype(int), 'TARGET': preds})
sub.to_csv('submissions/sub12.csv', index=False)